In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import torch
from torch import nn

In [2]:
# model_name = 'IDEA-CCNL/Wenzhong-GPT2-3.5B'
model_name = './model'

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [3]:
model = model.eval()

In [4]:
text = "北京位于"
encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [5]:
output = model(**encoded_input)

In [6]:
sample_output = model.generate(
    encoded_input['input_ids'],
    do_sample=True, 
    max_length=100, 
    top_k=50
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [7]:
tokenizer.decode(sample_output[0])

'北京位于北温带，地势由西北向东南倾斜。\n相对偏西的气流进入北京，会给降雨带来明显的增温作用，�'

In [17]:
encoded_input['attention_mask'].dtype

torch.int64

In [11]:
text = "北京位于南城的协和医院"
encoded_input = tokenizer(text, return_tensors='pt')
right_output = model(input_ids=encoded_input['input_ids'])
right_logits = right_output.logits[:, -1, :].view(-1)

In [16]:
right_logits.shape

torch.Size([50304])

In [61]:
encoded_input = tokenizer(text, return_tensors='pt')
diffs = []
for i in range(50):
    first_output = model(input_ids=torch.LongTensor([[i]]))
    second_output = model(input_ids=encoded_input['input_ids'], past_key_values=first_output.past_key_values)
    second_logits = second_output.logits[:, -1, :].view(-1)
    diff = (torch.sum((second_logits - right_logits) ** 2) / len(right_logits)).detach().item()
    print(i, diff)
    diffs.append((i, diff))
for i in range(tokenizer.vocab_size - 50, tokenizer.vocab_size):
    first_output = model(input_ids=torch.LongTensor([[i]]))
    second_output = model(input_ids=encoded_input['input_ids'], past_key_values=first_output.past_key_values)
    second_logits = second_output.logits[:, -1, :].view(-1)
    diff = (torch.sum((second_logits - right_logits) ** 2) / len(right_logits)).detach().item()
    print(i, diff)
    diffs.append((i, diff))

0 0.22758223116397858
1 0.42101413011550903
2 0.11645098030567169
3 0.21743199229240417
4 0.13183628022670746
5 0.08746851980686188
6 0.5519747734069824
7 0.7463037371635437
8 0.2039681226015091
9 0.15316486358642578
10 0.1427486538887024
11 0.15914595127105713
12 0.04127417132258415
13 0.41353127360343933
14 0.06513189524412155
15 0.17800942063331604
16 0.41627639532089233
17 0.13674382865428925
18 0.11395834386348724
19 0.12362097948789597
20 0.14189130067825317
21 0.12765437364578247
22 0.20745964348316193
23 0.09250973165035248
24 0.11203231662511826
25 0.2353578507900238
26 0.08013581484556198
27 0.5328487753868103
28 0.7309257984161377
29 0.25862255692481995
30 0.0751747414469719
31 0.15293598175048828
32 0.09560840576887131
33 0.09238307923078537
34 0.05506070330739021
35 0.0653311237692833
36 0.06226501241326332
37 0.0709947943687439
38 0.1473851054906845
39 0.09755463898181915
40 0.06169920414686203
41 0.07941905409097672
42 0.06480909883975983
43 0.07727842032909393
44 0.0865

In [62]:
diffs = sorted(diffs, key=lambda x: x[1])

In [67]:
print(diffs[:5])

[(12, 0.04127417132258415), (34, 0.05506070330739021), (40, 0.06169920414686203), (36, 0.06226501241326332), (42, 0.06480909883975983)]


In [71]:
ivocab = {v: k for k, v in tokenizer.get_vocab().items()}

In [84]:
ivocab[42]

'K'

In [85]:
first_output = model(input_ids=torch.LongTensor([[50256]]))

In [86]:
past_key_values = first_output.past_key_values
past_key_values = torch.stack([torch.stack(x) for x in past_key_values])
past_key_values = past_key_values.detach().numpy()
np.save('past_key_values', past_key_values)

In [10]:
class GPT(nn.Module):
    def __init__(self, model):
        super(GPT, self).__init__()
        self.model = model

    def forward(self, input_ids = None, past_key_values = None):
        if past_key_values is not None:
            past_key_values = torch.unbind(past_key_values)
            past_key_values = [torch.unbind(x) for x in past_key_values]
        out = self.model(input_ids=input_ids, past_key_values=past_key_values)
        # [20, 2, 1, 16, x, 64]
        # dim 4 is free
        past_key_values = out.past_key_values
        past_key_values = torch.stack([torch.stack(x) for x in past_key_values])

        return out.logits, past_key_values

In [11]:
gpt = GPT(model)

In [12]:
# first
output_seq = []
logits, past_key_values = gpt(input_ids=encoded_input['input_ids'])
next_token = int(np.argmax(logits[0, -1].detach().numpy()))
output_seq.append(next_token)
print(tokenizer.decode(output_seq))
for i in range(50):
    logits, past_key_values = gpt(input_ids=torch.LongTensor([[next_token]]), past_key_values=past_key_values)
    next_token = int(np.argmax(logits[0, -1].detach().numpy()))
    output_seq.append(next_token)
    print(tokenizer.decode(output_seq))

一
一�
一个
一个�
一个�
一个普
一个普�
一个普通
一个普通的
一个普通的人
一个普通的人�
一个普通的人�
一个普通的人，
一个普通的人，�
一个普通的人，我
一个普通的人，我的
一个普通的人，我的�
一个普通的人，我的名
一个普通的人，我的名�
一个普通的人，我的名字
一个普通的人，我的名字�
一个普通的人，我的名字叫
一个普通的人，我的名字叫AI
一个普通的人，我的名字叫AI�
一个普通的人，我的名字叫AI匠
一个普通的人，我的名字叫AI匠�
一个普通的人，我的名字叫AI匠�
一个普通的人，我的名字叫AI匠，
一个普通的人，我的名字叫AI匠，�
一个普通的人，我的名字叫AI匠，我
一个普通的人，我的名字叫AI匠，我的
一个普通的人，我的名字叫AI匠，我的�
一个普通的人，我的名字叫AI匠，我的名
一个普通的人，我的名字叫AI匠，我的名�
一个普通的人，我的名字叫AI匠，我的名字
一个普通的人，我的名字叫AI匠，我的名字�
一个普通的人，我的名字叫AI匠，我的名字叫
一个普通的人，我的名字叫AI匠，我的名字叫AI
一个普通的人，我的名字叫AI匠，我的名字叫AI�
一个普通的人，我的名字叫AI匠，我的名字叫AI匠
一个普通的人，我的名字叫AI匠，我的名字叫AI匠�
一个普通的人，我的名字叫AI匠，我的名字叫AI匠�
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，�
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我的
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我的�
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我的名
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我的名�
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我的名字
一个普通的人，我的名字叫AI匠，我的名字叫AI匠，我的名字�


In [13]:
past_key_values_start = torch.zeros([30, 2, 1, 32, 1, 96])
# first
output_seq = []
logits, past_key_values = gpt(input_ids=encoded_input['input_ids'], past_key_values=past_key_values_start)
next_token = int(np.argmax(logits[0, -1].detach().numpy()))
output_seq.append(next_token)
print(tokenizer.decode(output_seq))
for i in range(50):
    logits, past_key_values = gpt(input_ids=torch.LongTensor([[next_token]]), past_key_values=past_key_values)
    next_token = int(np.argmax(logits[0, -1].detach().numpy()))
    output_seq.append(next_token)
    print(tokenizer.decode(output_seq))

一
一�
一个
一个�
一个�
一个普
一个普�
一个普通
一个普通的
一个普通的人
一个普通的人�
一个普通的人�
一个普通的人，
一个普通的人，�
一个普通的人，我
一个普通的人，我的
一个普通的人，我的�
一个普通的人，我的名
一个普通的人，我的名�
一个普通的人，我的名字
一个普通的人，我的名字�
一个普通的人，我的名字叫
一个普通的人，我的名字叫AI
一个普通的人，我的名字叫AI�
一个普通的人，我的名字叫AI匠
一个普通的人，我的名字叫AI匠�
一个普通的人，我的名字叫AI匠�
一个普通的人，我的名字叫AI匠，
一个普通的人，我的名字叫AI匠，�
一个普通的人，我的名字叫AI匠，我
一个普通的人，我的名字叫AI匠，我是
一个普通的人，我的名字叫AI匠，我是一
一个普通的人，我的名字叫AI匠，我是一�
一个普通的人，我的名字叫AI匠，我是一个
一个普通的人，我的名字叫AI匠，我是一个�
一个普通的人，我的名字叫AI匠，我是一个�
一个普通的人，我的名字叫AI匠，我是一个普
一个普通的人，我的名字叫AI匠，我是一个普�
一个普通的人，我的名字叫AI匠，我是一个普通
一个普通的人，我的名字叫AI匠，我是一个普通的
一个普通的人，我的名字叫AI匠，我是一个普通的人
一个普通的人，我的名字叫AI匠，我是一个普通的人�
一个普通的人，我的名字叫AI匠，我是一个普通的人�
一个普通的人，我的名字叫AI匠，我是一个普通的人，
一个普通的人，我的名字叫AI匠，我是一个普通的人，�
一个普通的人，我的名字叫AI匠，我是一个普通的人，我
一个普通的人，我的名字叫AI匠，我是一个普通的人，我的
一个普通的人，我的名字叫AI匠，我是一个普通的人，我的�
一个普通的人，我的名字叫AI匠，我是一个普通的人，我的名
一个普通的人，我的名字叫AI匠，我是一个普通的人，我的名�
一个普通的人，我的名字叫AI匠，我是一个普通的人，我的名字


In [14]:
print('start export')
!rm -rf onnx && mkdir -p onnx
input_ids = torch.LongTensor([[1]])
torch.onnx.export(
    gpt,               # model being run
    (input_ids, past_key_values_start),                         # model input (or a tuple for multiple inputs)
    "onnx/model.onnx",   # where to save the model (can be a file or file-like object)
    export_params=True,        # store the trained parameter weights inside the model file
    opset_version=13,          # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names = ['input', 'pkv'],   # the model's input names
    output_names = ['output', 'pkv_output'], # the model's output names
    dynamic_axes={
        'input' : {0 : 'batch_size', 1 : 'seq_len'},    # variable length axes
        'output' : {0 : 'batch_size', 1 : 'seq_len'},
        'pkv': {4: 'seq_len'},
        'pkv_output': {4: 'seq_len'},
    }
)
print('done')

start export


/opt/conda/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py:196: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  attn_weights = attn_weights / (float(value.size(-1)) ** 0.5)


done


In [19]:
from onnxruntime.quantization import quantize_dynamic, QuantType

In [20]:
!rm -rf onnxq && mkdir -p onnxq
model_fp32 = 'onnx/model.onnx'
model_quant = 'onnxq/model.onnx'
quantized_model = quantize_dynamic(
    model_fp32,
    model_quant,
    use_external_data_format=True,
    weight_type=QuantType.QUInt8,  # 默认的QInt8会产生极大的误差
    extra_options={
        'DisableShapeInference': True,
    }
)

Ignore MatMul due to non constant B: /[MatMul_346]
Ignore MatMul due to non constant B: /[MatMul_370]
Ignore MatMul due to non constant B: /[MatMul_546]
Ignore MatMul due to non constant B: /[MatMul_570]
Ignore MatMul due to non constant B: /[MatMul_746]
Ignore MatMul due to non constant B: /[MatMul_770]
Ignore MatMul due to non constant B: /[MatMul_946]
Ignore MatMul due to non constant B: /[MatMul_970]
Ignore MatMul due to non constant B: /[MatMul_1146]
Ignore MatMul due to non constant B: /[MatMul_1170]
Ignore MatMul due to non constant B: /[MatMul_1346]
Ignore MatMul due to non constant B: /[MatMul_1370]
Ignore MatMul due to non constant B: /[MatMul_1546]
Ignore MatMul due to non constant B: /[MatMul_1570]
Ignore MatMul due to non constant B: /[MatMul_1746]
Ignore MatMul due to non constant B: /[MatMul_1770]
Ignore MatMul due to non constant B: /[MatMul_1946]
Ignore MatMul due to non constant B: /[MatMul_1970]
Ignore MatMul due to non constant B: /[MatMul_2146]
Ignore MatMul due to

In [21]:
!du -sh onnx

14G	onnx


In [22]:
!du -sh onnxq

3.5G	onnxq


In [ ]:
!rm -rf onnx